In [ ]:
import pandas as pd
import numpy as np
# %matplotlib notebook
import matplotlib.pyplot as plt
from collections import defaultdict

In [2]:
from sklearn.linear_model import LinearRegression, RANSACRegressor, TheilSenRegressor

In [3]:
df = pd.read_csv('entire_input.csv')
pd.set_option('display.max_columns', None)


In [4]:
df

In [31]:
state = dict()
xs = []
ys = []
for i, row in df.iterrows():
    #print(row)
    session = (row['ip_src'], row['ip_dst'], row['tcp_src'], row['tcp_dst'])
    
    if session in state and row['ip_src'] != 3232236018:
        xs.append(state[session])
        ys.append(row['ip_id'])
    state[session] = row['ip_id']
xs = np.array(xs).reshape(-1, 1)
ys = np.array(ys).reshape(-1, 1)

In [ ]:
state_noacanthisitta = dict()
state2 = dict()
state3 = dict()
state4 = dict()
xs_noacanthisitta = []
ys_noacanthisitta = []
for i, row in df.iterrows():
    #print(row)
    session = (row['ip_src'], row['ip_dst'], row['tcp_src'], row['tcp_dst'])
    
    if session in state_noacanthisitta and not(row['tcp_ack'] == 1 and row['tcp_syn'] == 1) and not(state2[session] == 1 and state3[session] == 1) and not state4[session] == 1:
        xs_noacanthisitta.append(state_noacanthisitta[session])
        ys_noacanthisitta.append(row['ip_id'])
    state_noacanthisitta[session] = row['ip_id']
    state2[session] = row['tcp_syn']
    state3[session] = row['tcp_ack']
    state4[session] = row['tcp_rst']
    
xs_noacanthisitta = np.array(xs_noacanthisitta).reshape(-1, 1)
ys_noacanthisitta = np.array(ys_noacanthisitta).reshape(-1, 1)

In [43]:
# 0,0: ipid = 0, syn = 1, ack = 1
# 1,0: ipid != 0, syn = 1, ack = 1
# 0,1: ipid = 0, syn = 0 || ack = 0
# 1,1: ipid != 0, syn = 0 || ack = 0
squares = np.zeros((2,2))
for i, row in df.iterrows():
    #print(row)
    #if row['ip_src'] == 3232236018: # 192.168.1.242 = acanthisitta, known to be linux
    if row['ip_id'] == 0 and row['tcp_syn'] == 1 and row['tcp_ack'] == 1:
        squares[0, 0] += 1
    elif row['ip_id'] != 0 and row['tcp_syn'] == 1 and row['tcp_ack'] == 1:
        squares[1, 0] += 1
    elif row['ip_id'] == 0 and (row['tcp_syn'] == 0 or row['tcp_ack'] == 0):
        squares[0, 1] += 1
    elif row['ip_id'] != 0 and (row['tcp_syn'] == 0 or row['tcp_ack'] == 0):
        squares[1, 1] += 1


In [5]:
state_ack_no = dict()
base_seq = dict()
base_ack = dict()
xs_ack_no = []
ys_ack_no = []
for i, row in df.iterrows():
    #print(row)
    session = (row['ip_src'], row['ip_dst'], row['tcp_src'], row['tcp_dst'])
    reverse_session = (row['ip_dst'], row['ip_src'], row['tcp_dst'], row['tcp_src'])
    
    if session not in base_ack and row['tcp_ack_no'] != 0:
        base_ack[session] = [row['tcp_ack_no']]
    if session not in base_seq:
        base_seq[session] = [row['tcp_seq_no']]
    if session in state_ack_no and reverse_session in state_ack_no and reverse_session in base_ack and reverse_session in base_seq and session in base_ack:
        xs_ack_no.append(state_ack_no[session] + state_ack_no[reverse_session])# + base_seq[session] + base_seq[reverse_session] + base_ack[session] + base_ack[reverse_session])[[0,1,2,3,4,5,6,7,8,9]])
        #print(state_ack_no[session] + state_ack_no[reverse_session] + base_seq[session] + base_seq[reverse_session] + base_ack[session] + base_ack[reverse_session])
        
        if session in base_ack:
            ys_ack_no.append(row['tcp_ack_no'] - base_ack[session])
        else:
            ys_ack_no.append(0) #lets pray
            #ys_ack_no.append(row['tcp_ack_no'])
        state_ack_no[session] = [row['tcp_ack_no'] - base_ack[session][0], row['tcp_seq_no'] - base_seq[session][0], row['tcp_data_off'], row['len_original']]
    if session not in state_ack_no :
        if session in base_ack:
            state_ack_no[session] = [row['tcp_ack_no'] - base_ack[session][0], row['tcp_seq_no'] - base_seq[session][0], row['tcp_data_off'], row['len_original']]
        else:
            state_ack_no[session] = [0, 0, row['tcp_data_off'], row['len_original']]

xs_ack_no = np.array(xs_ack_no)
ys_ack_no = np.array(ys_ack_no).reshape(-1, 1)

In [ ]:
state_ack_no = dict()
sent_acks = defaultdict(lambda: set())
received_acks = defaultdict(lambda: set())
base_seq = dict()
base_ack = dict()
xs_ack_no = []
ys_ack_no = []
wrong = 0
discarded = 0
for i, row in df.iterrows():
    #print(row)

    session = (row['ip_src'], row['ip_dst'], row['tcp_src'], row['tcp_dst'])
    reverse_session = (row['ip_dst'], row['ip_src'], row['tcp_dst'], row['tcp_src'])
    if row['tcp_ack_no'] in sent_acks[session]:
        ...
    if session in state_ack_no and \
            ((row['tcp_fin'] == 0 and row['tcp_rst'] == 0 and (row['tcp_ack'] == 0 or row['tcp_syn'] == 1)) or \
          (row['tcp_fin'] == 1 and row['tcp_ack'] == 1) or \
            (row['tcp_rst'] == 1 and row['tcp_ack'] == 1) or
            (row['tcp_ack'] == 1 and row['tcp_syn'] == 0 and row['tcp_rst'] == 0 and row['tcp_fin'] == 0 and 
        (row['tcp_ack_no'], row['tcp_seq_no']) not in sent_acks[session])): # and reverse_session in state_ack_no:

        xs_ack_no.append(state_ack_no[session])# + state_ack_no[reverse_session])
        #print(state_ack_no[session] + state_ack_no[reverse_session] + base_seq[session] + base_seq[reverse_session] + base_ack[session] + base_ack[reverse_session])
        ys_ack_no.append(row['tcp_seq_no'])
        if row['tcp_seq_no'] != sum(state_ack_no[session]):
            wrong += 1
    elif session in state_ack_no and row['tcp_seq_no'] == sum(state_ack_no[session]):
        discarded += 1
        
    state_ack_no[session] = [row['tcp_seq_no'], row['tcp_syn'], row['payload_size']]
    if row['tcp_fin'] == 1 or row['tcp_rst'] == 1:
        state_ack_no.pop(session)
        sent_acks.pop(reverse_session, None)
    if row['tcp_ack'] == 1 and row['tcp_syn'] == 0 and row['payload_size'] > 0:
        sent_acks[session].add((row['tcp_ack_no'], row['tcp_seq_no']))
        received_acks[session].add(row['tcp_ack_no'])



xs_ack_no = np.array(xs_ack_no)
ys_ack_no = np.array(ys_ack_no)

In [ ]:
discarded

In [ ]:
wrong

In [144]:
fig, ax = plt.subplots()
im = ax.imshow(squares)
ax.text(0, 0, squares[0, 0], ha="center", va="center", color="w")
ax.text(0, 1, squares[1, 0], ha="center", va="center", color="c")
ax.text(1, 0, squares[0, 1], ha="center", va="center", color="m")
ax.text(1, 1, squares[1, 1], ha="center", va="center", color="y")

In [ ]:
df

In [41]:
cnt

In [50]:
ys_ack_no

In [51]:
xs_ack_no = xs_ack_no / [2.**32, 1., 2.**16]
ys_ack_no = ys_ack_no / 2.**16

In [ ]:
ys_ack_no = ys_ack_no / 2.**16

In [ ]:
np.min(xs_ack_no, axis=0)

In [7]:
r = RANSACRegressor()

In [35]:
r.fit(xs_ack_no, ys_ack_no)

In [ ]:
(1.52030289e-05) ** 2

In [ ]:
r.coef_ 

In [ ]:
d = (xs_ack_no @ np.array([1, 1, 1]).reshape(3,) - ys_ack_no)

In [ ]:
plt.figure()
plt.plot(np.log10(abs(d * 2**32)+1), '.-')

In [36]:
ys2 = r.predict(xs_ack_no)

In [24]:
sum(d > 0)

In [ ]:
ys2

In [46]:
plt.scatter(xs_ack_no[:, 0], ys_ack_no, alpha=0.01) 
plt.scatter(xs_ack_no[:, 2], ys2, alpha=0.01)


In [288]:
rr = np.unique_counts(ys_ack_no - ys2)

In [326]:
plt.hist(ys_ack_no - ys2, bins=201, range=(-1e7, 1e7))

In [214]:
v = list(zip(rr.values, rr.counts))